<a href="https://colab.research.google.com/github/Hifjur/Twitter-Sentiment-Analysis/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset from 
https://www.kaggle.com/cosmos98/twitter-and-reddit-sentimental-analysis-dataset

In [1]:
#importing the libraries
import tweepy
import pandas as pd
import numpy as np
import gensim
import os
import nltk
nltk.download('punkt')
import matplotlib.pyplot as plt
import csv


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#Load Dataset
from google.colab import files
uploaded = files.upload()


Saving Twitter_Data.csv to Twitter_Data.csv


In [2]:
#loading data
df = pd.read_csv('https://raw.githubusercontent.com/Hifjur/Twitter-Sentiment-Analysis/main/Twitter_Data.csv')


In [3]:
df.head()

,clean_text,category
0,when modi promised “minimum government maximum...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0


In [ ]:

# to string using join() function
    
# Function to convert  
def listToString(s): 
    
    # initialize an empty string
    str1 = " " 
    
    # return string  
    return (str1.join(s))
        

In [ ]:
#Read the data and parsing to get the data ready for training the model
sentences = []
text = ''
def filter_words(sentence):
    return [word for word in nltk.word_tokenize(sentence) if word.isalnum()]

def tokenize(text):
    return [filter_words(sentence) for sentence in nltk.sent_tokenize(text.lower())]
with open("Twitter_Data.csv", 'r') as f:
    f_csv = csv.reader(f)
    for line in f_csv:
        a0 = line
        text = text +' '+ listToString(a0)
        print(a0)
    sentences.extend(tokenize(text))

In [ ]:
# create the model and tune it!
word2Vec_model = gensim.models.Word2Vec(
                    sentences, # input sentences
                    sg=1, # use skip grams (0=CBOW). skip grams tend to work better on smaller corpora
                    size=100, # this is how many dimensions the vectors will be
                    min_count=1 # how many times must a word apper
                    )
# Let's save the model to file!
word2Vec_model.save('my_vecs.p')

In [ ]:
from google.colab import files
files.download('my_vecs.p') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#m = gensim.models.Word2Vec.load('my_vecs.p')
#voc =m.wv.vocab
#print(voc.)
from sklearn.decomposition import PCA

my_model = gensim.models.Word2Vec.load('my_vecs.p')

model_vocab = my_model.wv.vocab

pca = PCA(n_components=2)

my_pca = pca.fit_transform(my_model[model_vocab])

plt.scatter(my_pca[:,0], my_pca[:,1])

for i, word in enumerate(model_vocab):
    plt.annotate(word, xy=(my_pca[i,0], my_pca[i,1]))
    print(word)

In [ ]:
print(word2Vec_model.wv.most_similar('poor'))

[('not', 0.9983155727386475), ('they', 0.9982820153236389), ('1', 0.9982790946960449), ('you', 0.9982743263244629), ('modi', 0.9982554316520691), ('your', 0.9982263445854187), ('has', 0.9982134699821472), ('but', 0.998192548751831), ('dont', 0.9981778860092163), ('who', 0.9981662631034851)]


In [ ]:
print(word2Vec_model.wv.similarity('food', 'election')) 

0.9418461


In [ ]:
#Visualisation
from sklearn.decomposition import PCA

my_model = gensim.models.Word2Vec.load('my_vecs.p')

model_vocab = my_model.wv.vocab

pca = PCA(n_components=2)

my_pca = pca.fit_transform(my_model[model_vocab])

plt.scatter(my_pca[:,0], my_pca[:,1])

for i, word in enumerate(model_vocab):
    plt.annotate(word, xy=(my_pca[i,0], my_pca[i,1]))

plt.show()

In [4]:
#importing libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

Converting to vector


In [15]:
#TFIDF vector
X = df['clean_text']
tfidf = TfidfVectorizer(max_features=10000, ngram_range=(1,2))
X = tfidf.fit_transform(X.values.astype(str))
Y = df['category']

In [6]:
X.shape

(162980, 10000)

In [7]:
#getting rid of null values
df.dropna(inplace= True)

In [8]:
#train the data set 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [19]:
from sklearn.metrics import accuracy_score

#accuracy_score(Y_train,Y_test)

In [20]:
X

<162969x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 3483712 stored elements in Compressed Sparse Row format>

In [9]:
X_train.shape, X_test.shape

((130384, 10000), (32596, 10000))

In [10]:
df.isnull().sum()

clean_text    0
category      0
dtype: int64

In [11]:
df['category'].isnull().values.any()

False

In [12]:
df['clean_text'].isnull().values.any()

False

In [32]:
from sklearn.preprocessing import StandardScaler

In [41]:
clf = LinearSVC()
#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)
#np.where(X_train.all() >= np.finfo(np.float64).max)
clf.fit(X_train, Y_train)

ValueError: ignored